In [21]:
%pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In the notebook eda, I did some simple work with the mortality dataset and then used the NPS API to get park codes that matched the park name for each death. Now, I'm going to start with that dataset to get even more info from the NPS API.

In [ ]:
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import requests, time

from dotenv import load_dotenv

import os

Load API key from .env

In [26]:
load_dotenv()

API_KEY = os.getenv("NPS_API_KEY")

print(API_KEY is not None)

True


In [10]:
df = pd.read_csv("../data/mortality_with_parks.csv")

df.head()

,Incident Date,Park Name,Cause of Death,Cause of Death Group \n(Used in the NPS Mortality Dashboard),Intent,Outcome,Sex,Age Range,Activity,parkCode,fullName_official,skip_api
0,2007-01-01,Glen Canyon National Recreation Area,Undetermined,Undetermined,Undetermined,Fatal injury,Male,65+,Not Reported,glca,Glen Canyon National Recreation Area,False
1,2007-01-22,Golden Gate National Recreation Area,Drowning,Drowning,Unintentional,Fatal injury,Male,Not Reported,Vessel Related,goga,Golden Gate National Recreation Area,False
2,2007-01-22,Golden Gate National Recreation Area,Undetermined,Undetermined,Undetermined,Fatal injury,Male,Not Reported,Vessel Related,goga,Golden Gate National Recreation Area,False
3,2007-01-29,Natchez Trace Parkway,Motor Vehicle Crash,Motor Vehicle Crash,Unintentional,Fatal injury,Female,15-24,Driving,natr,Natchez Trace Parkway,False
4,2007-01-29,Natchez Trace Parkway,Motor Vehicle Crash,Motor Vehicle Crash,Unintentional,Fatal injury,Female,45-54,Driving,natr,Natchez Trace Parkway,False


I need the year of each event to lookup the visitor statistics for that year. I'm going to pull it out of the incident date column. I want some more info about that column first.

In [11]:
print(df["Incident Date"].dtype)

object


The date is currently an object -- a string in this case -- so I'm going to convert it to datetime.

In [13]:
df["Incident Date"] = pd.to_datetime(df["Incident Date"], errors="coerce")


run this line again to confirm that it was changed correctly.

In [14]:
print(df["Incident Date"].dtype)

datetime64[ns]


Check to see if there are any missing dates.

In [15]:
print(df["Incident Date"].isna().sum())

0


Next, I'm going to build the dataframe I need to request from the API -- that park code and incident year for every row that has a park code, and no rows that do not.

In [ ]:
pairs = (
    #filter out rows that don't have a park code
    df.loc[df["parkCode"].notna(), ["parkCode", "Incident Date"]]
    #Pull the calendar year into a new column to search for annual data
      .assign(year=lambda d: d["Incident Date"].dt.year.astype(int))
    #drop duplicates since I can match data to all lines with the same code and year
      .drop_duplicates(subset=["parkCode", "year"])
      .reset_index(drop=True)
)

#Print length of the new dataframe and the first few lines to make sure the info is correct
print("Unique (parkCode, year) pairs:", len(pairs))
pairs.head()

Unique (parkCode, year) pairs: 1340


,parkCode,Incident Date,year
0,glca,2007-01-01,2007
1,goga,2007-01-22,2007
2,natr,2007-01-29,2007
3,gate,2007-02-07,2007
4,blca,2007-02-10,2007
